In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
print(tf.VERSION)
print(tf.__version__)

In [ ]:
cora_content = pd.read_csv('./cora/cora.content', sep='\t', header=None)
cora_content.head()

In [ ]:
ids = cora_content[0].values # paper(node) ids
vecs = cora_content[cora_content.columns[1:1434]].values # node features
labels = cora_content[1434].values # node label

print(np.unique(labels))

In [ ]:
# node label one hot encoding
labels_onehot = LabelEncoder().fit_transform(labels)
labels_onehot = np.expand_dims(labels_onehot, axis=1)
labels_onehot = OneHotEncoder().fit_transform(labels_onehot).toarray()

In [ ]:
inds = np.arange(ids.shape[0]) # use index at identifying each node
x = vecs
y = labels_onehot
print(ids.shape, x.shape, y.shape)

In [ ]:
num_classes = 7
num_per_train = 10
num_per_test = 100
x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split(x, y, inds, stratify=y,
                                                    train_size=num_classes*num_per_train,
                                                    test_size=num_classes*num_per_test,
                                                    random_state=42)

x_train, x_valid, y_train, y_valid, idx_train, idx_valid = train_test_split(x_train, y_train, idx_train,
                                                      stratify=y_train,
                                                      train_size=int(num_classes*num_per_train*0.8),
                                                      test_size=int(num_classes*num_per_train*0.2),
                                                      random_state=42)

print(idx_train.shape, x_train.shape, y_train.shape) # 10 examples per class
print(idx_valid.shape, x_valid.shape, y_valid.shape) # 10 examples per class
print(idx_test.shape, x_test.shape, y_test.shape) # 100 examples per class

## model

In [ ]:
class DNN():
    def __init__(self, input_dim=1433, num_classes=7):
        init = tf.initializers.he_normal()
        
        self.x = tf.placeholder(tf.float32, [None, input_dim])
        self.y = tf.placeholder(tf.float32, [None, num_classes])
        
        self.W1 = tf.Variable(init([input_dim, 128]))
        self.b1 = tf.Variable(tf.zeros([128]), tf.float32)
        self.L1 = tf.matmul(self.x, self.W1) + self.b1
        self.L1 = tf.nn.relu(self.L1)
        
        self.W2 = tf.Variable(init([128, 128]))
        self.b2 = tf.Variable(tf.zeros([128]), tf.float32)
        self.L2 = tf.matmul(self.L1, self.W2) + self.b2
        self.L2 = tf.nn.relu(self.L2)
        
        self.W3 = tf.Variable(init([128, num_classes]))
        self.b3 = tf.Variable(tf.zeros([num_classes]), tf.float32)
        self.logit = tf.matmul(self.L2, self.W3) + self.b3
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logit,
                                                                              labels=self.y))
        
        optimizer = tf.train.AdamOptimizer()
        self.train_step = optimizer.minimize(self.loss)
        
        self.pred = tf.argmax(self.logit, axis=1)
        
        correct = tf.equal(self.pred, tf.argmax(self.y, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
# train and evaluate dnn
def dnn():
    model = DNN()
    epochs = 20
    train_step = model.train_step
    loss = model.loss
    accuracy = model.accuracy
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            # full batch training because full batch size is 70(small)
            _, train_loss, train_acc = sess.run([train_step, loss, accuracy], feed_dict={
                model.x: x_train,
                model.y: y_train
            })
            
            _, valid_loss, valid_acc = sess.run([train_step, loss, accuracy], feed_dict={
                model.x: x_valid,
                model.y: y_valid
            })
            
            print('epoch: %2d, tr_loss: %.4f, tr_acc: %.4f, val_loss: %.4f, val_acc: %.4f' %
                  (epoch, train_loss, train_acc, valid_loss, valid_acc))
            
            
        l, a = sess.run([loss, accuracy], feed_dict={
            model.x: x_test,
            model.y: y_test
        })
        
        print('test loss: %.4f, accuracy: %.4f' % (l, a))
dnn()

In [ ]:
class GCN():
    def __init__(self, input_dim=1433, num_classes=7, num_nodes=2708):
        self.x = tf.placeholder(tf.float32, [num_nodes, input_dim])
        self.y = tf.placeholder(tf.float32, [num_nodes, num_classes])
        self.A = tf.placeholder(tf.float32, [num_nodes, num_nodes])
        self.mask = tf.placeholder(tf.int32, [None, 1]) # masking for train/valid/test
        
        init = tf.initializers.he_normal()
        
        self.W1 = tf.Variable(init([input_dim, 128]))
        self.L1 = tf.matmul(tf.matmul(self.A, self.x), self.W1)
        self.L1 = tf.nn.tanh(self.L1)
        
        self.W2 = tf.Variable(init([128, num_classes]))
        self.L2 = tf.matmul(tf.matmul(self.A, self.L1), self.W2)
        self.L2 = tf.nn.relu(self.L2)
        
        logits = tf.gather_nd(self.L2, self.mask)
        labels = tf.gather_nd(self.y, self.mask)
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                              labels=labels))
        
        optimizer = tf.train.AdamOptimizer()
        self.train_step = optimizer.minimize(self.loss)
        
        correct = tf.equal(tf.argmax(logits, axis=1), tf.argmax(labels, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        

In [ ]:
# make adj matrix from citation information
def get_adj_matrix(ids):
    cora_cites = np.loadtxt('./cora/cora.cites', dtype=np.int32)
    N = ids.shape[0]
    adj_matrix = np.zeros(shape=(N, N), dtype=np.int32)
    
    # iterate over line
    for i in range(cora_cites.shape[0]):
        node1, node2 = cora_cites[i]
        idx1 = np.where(ids==node1)[0]
        idx2 = np.where(ids==node2)[0]
        
        # treat as undirected graph
        adj_matrix[idx1, idx2] = 1
        adj_matrix[idx2, idx1] = 1
    
    return adj_matrix
    
# make DAD(normalization) matrix
def get_norm_matrix(adj_matrix):
    a_tilda = adj_matrix + np.eye(adj_matrix.shape[0]) # A_ = A+I
    d_tilda = np.diag(1 / np.sqrt(np.sum(a_tilda, axis=1))) # D_^(-1/2)
    return np.matmul(np.matmul(d_tilda, a_tilda), d_tilda)

In [ ]:
def gcn():
    epochs = 20
    model = GCN()
    loss = model.loss
    train_step = model.train_step
    accuracy = model.accuracy
    
    adj_matrix = get_adj_matrix(ids)
    norm_matrix = get_norm_matrix(adj_matrix)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            # train set
            _, train_loss, train_acc = sess.run([train_step, loss, accuracy], feed_dict={
                model.x: x,
                model.y: y,
                model.A: norm_matrix,
                model.mask: np.expand_dims(idx_train, axis=1)
            })
            
            # valid set
            _, valid_loss, valid_acc = sess.run([train_step, loss, accuracy], feed_dict={
                model.x: x,
                model.y: y,
                model.A: norm_matrix,
                model.mask: np.expand_dims(idx_valid, axis=1)
            })
            print('epoch: %2d, train_loss: %.4f, train_acc: %.4f, valid_loss: %.4f, valid_acc: %.4f' %
                 (epoch, train_loss, train_acc, valid_loss, valid_acc))
    
        # test set
        _, test_loss, test_acc = sess.run([train_step, loss, accuracy], feed_dict={
            model.x: x,
            model.y: y,
            model.A: norm_matrix,
            model.mask: np.expand_dims(idx_test, axis=1)
        })
        
        print('test_loss: %.4f, test_acc: %.4f' % (test_loss, test_acc))
    
gcn()